# Preprocesamiento de datos

In [97]:
import pandas as pd
import os

url = "C:/Users/USUARIO/Desktop/Proyectos/Sistema_recomendacion/data/raw/data_raw.csv"

if os.path.exists(url):
    data = pd.read_csv(url, sep=',', encoding='latin-1')
    print("Datos cargados exitosamente.", data.shape)
else:
    print("Archivo no encontrado. Verifica la ruta.")

Datos cargados exitosamente. (51290, 24)


## Preprocesamiento

In [98]:
data_clean = data.copy()

### Cambiando tipo de datos

In [99]:
data_clean["Order Date"] = pd.to_datetime(data_clean["Order Date"], format="%d-%m-%Y")
data_clean["Ship Date"] = pd.to_datetime(data_clean["Ship Date"], format="%d-%m-%Y")

### Eliminar columnas

#### Análisis de la columna Postal Code

La columna 'Postal Code' presenta una gran cantidad de valores faltantes. Como se indicó en el análisis exploratorio de datos (EDA), la recomendación inicial es eliminar la columna. Sin embargo, para asegurar una buena decisión, se evalua la variabilidad de esta característica, ya que cuenta aún con '9994' datos no faltantes. Si estos datos corresponen a diferentes clientes, se entrenará y evaluará uno modelo de sistema de recomendación utilizando únicamente los registros de los clientes con esta información. Si por el contrario no aporta información se descartará por completo esta columna.

In [100]:
# Calcular el porcentaje de valores faltantes
missing_values = data_clean.isna().sum()
missing_percentage = (missing_values["Postal Code"] / len(data_clean)) * 100
print(f"El porcentaje de datos faltantes en la columna Postal Code es de {missing_percentage:.2f}%")

El porcentaje de datos faltantes en la columna Postal Code es de 80.51%


In [101]:
postal_notna = data_clean[data_clean["Postal Code"].notna()]
clientes_postal = len(postal_notna["Customer ID"].unique()) / len(data_clean["Customer ID"].unique()) * 100
print("Tamaño de dataset con los valores no faltantes de Postal Code", postal_notna.shape)
print(f"Porcentaje de clientes con código postal {clientes_postal:.2f}%")

Tamaño de dataset con los valores no faltantes de Postal Code (9994, 24)
Porcentaje de clientes con código postal 49.87%


La columna 'Postal Code' tiene una proporción muy alta de datos faltantes de 80.51% y solo alrededor del 50% de los clientes cuentan con código postal, al seleccionar solo los clientes que no tengan datos faltantes en esta columna se pierde más de la mitad de los datos. 
La decisión es eliminar por completo la columna 'Postal Code', ya que no aporta un valor crucial para el análisis. Ya que, se cuenta con información geográfica suficiente mediante las columnas 'Country', 'State' y 'City'.
Como se cuenta con 

Las siguientes columnas sera eliminadas del dataset:

* 'Row ID', 'Order ID'. Ya que, no se consideran útiles para el sistema de recomendación al ser identificadores únicos. 
* 'Customer Name', porque es información personal que no contribuye al análisis de comportamiento de compra.
* 'Ship Date', ya que recomendar productos, con base a la rapidez de entrega o tiempo de envio no hace parte del objetivo del sistema de recomendación.
* 'Order Priority', No se considera que la prioridad de los pedidos afecten la recomendaciones.
* 'Product Name', similar a 'Customer Name'.

In [102]:
data_clean.drop(columns=["Row ID", 
                         "Order ID", 
                         "Postal Code", 
                         "Customer Name", 
                         "Ship Date", 
                         "Order Priority", 
                         "Product Name"], 
                inplace=True)

### Conjunto de datos para un sistema de recomendación basado en contenido
El siguiente dataset se construye teniendo como criterio que el sistema de recomendación estará basado en contenido, donde las características que describen los productos son relevantes. Por ende, se eliminaran más columnas, que no son importantes para este enfoque.

In [103]:
data_contenido = data_clean.copy()

In [104]:
data_contenido.drop(columns=["Order Date", 
                             "Ship Mode", 
                             "Customer ID",
                             "Segment", 
                             "City", 
                             "State", 
                             "Country", 
                             "Market",
                             "Region",
                             "Shipping Cost"], inplace=True)

In [105]:
data_contenido

,Product ID,Category,Sub-Category,Sales,Quantity,Discount,Profit
0,TEC-AC-10003033,Technology,Accessories,2309.650,7,0.0,762.1845
1,FUR-CH-10003950,Furniture,Chairs,3709.395,9,0.1,-288.7650
2,TEC-PH-10004664,Technology,Phones,5175.171,9,0.1,919.9710
3,TEC-PH-10004583,Technology,Phones,2892.510,5,0.1,-96.5400
4,TEC-SHA-10000501,Technology,Copiers,2832.960,8,0.0,311.5200
...,...,...,...,...,...,...,...
51285,OFF-FA-10000746,Office Supplies,Fasteners,65.100,5,0.0,4.5000
51286,OFF-AP-10002906,Office Supplies,Appliances,0.444,1,0.8,-1.1100
51287,OFF-EN-10001219,Office Supplies,Envelopes,22.920,3,0.0,11.2308
51288,OFF-BI-10000806,Office Supplies,Binders,13.440,2,0.0,2.4000


#### Codificar variables categóricas nominales
Se codificaran las variables categóricas mediante la técnica BinaryEncoder debidó a la gran cantidad de categorías presentes.

In [ ]:
import category_encoders as ce

binary_encoder = ce.BinaryEncoder(cols=["Product ID", "Category", "Sub-Category"])
df_cualitativas_encoded = binary_encoder.fit_transform(data_contenido)


KeyboardInterrupt: 

In [109]:
from sklearn.neighbors import NearestNeighbors

nbrs = NearestNeighbors(n_neighbors=20, radius=0.2)
nbrs.fit(data_contenido)

ValueError: could not convert string to float: 'TEC-AC-10003033'